In [6]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 816.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 7.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 2.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 2.7 MB/s eta 0:00:0000:0100:01


In [23]:
from datasets import load_dataset, Dataset
from transformers import AutoModelForSequenceClassification, AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np
import pandas as pd
from torch.nn.functional import softmax
from sentence_transformers import CrossEncoder
from tqdm.notebook import tqdm
from huggingface_hub import login

In [2]:
classifier_checkpoint="s-nlp/roberta_toxicity_classifier"
paraphraser_checkpoint="dphn/dolphin-2.9.3-mistral-nemo-12b"
similarity_checkpoint="cross-encoder/stsb-roberta-large"
dataset="SetFit/toxic_conversations"
torch.manual_seed(42)

In [3]:
classifier=AutoModelForSequenceClassification.from_pretrained(classifier_checkpoint, dtype=torch.bfloat16, device_map="auto")
paraphraser=AutoModelForCausalLM.from_pretrained(paraphraser_checkpoint, dtype=torch.bfloat16, device_map="auto")
similarity=CrossEncoder(similarity_checkpoint, model_kwargs={"dtype": torch.bfloat16})
classifier_tokenizer=AutoTokenizer.from_pretrained(classifier_checkpoint)
paraphraser_tokenizer=AutoTokenizer.from_pretrained(paraphraser_checkpoint, fix_mistral_regex=True)

Some weights of the model checkpoint at s-nlp/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
print(classifier.device, paraphraser.device, similarity.device)

cuda:0 cuda:0 cuda:0


In [5]:
ds=load_dataset(dataset, split="test")
ds=ds.filter(lambda ex: ex["label"]==1)
ds

Repo card metadata block was not found. Setting CardData to empty.


Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'label_text'],
    num_rows: 3954
})

In [6]:
ex=ds[0]
sentence=ex["text"]
label=ex["label"]
print(sentence, label)

stop making excuse for your infantile and stupid behavior.  there are ways to express your displeasure but showing your finger is not one of these ways.  you have disrespected the process and brought shame to the Hawaii delegation and people.   you owe the people of Hawaii an apology and to the DNC.  If you refuse to apologize then you should resign your position with the Hawaii democratic party.  there is no bravery involved just plain stupidity.   you are stooping low like trump is doing in his immature behavior when people disagree with him.   maybe you should join the repubs, you should feel at home with them.  dems do not tolerate stupidity.   no fd, she does not love Hawaii, she lovers herself, if she did she would not have done what she did to bring shame to us. 1


In [7]:
system_prompt="""You are a linguistic transformation agent. Your task is to analyze an input sentence and rewrite it by shifting its tone and toxicity level while maintaining 100% semantic equivalence.
Rules:
1. Analysis: Identify the current tone (e.g., polite, neutral, aggressive) and toxicity level.
2. Transformation: If the input is civil, rewrite it to be toxic/hostile. If the input is toxic, rewrite it to be civil/professional.
3. Meaning: Do not change the core facts or the intent of the message.
4. Output Format: Provide only the paraphrased sentence."""

In [11]:
    prompt=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": sentence}
    ]
    tokenized_prompt=paraphraser_tokenizer.apply_chat_template(prompt,
                                                   add_generation_prompt=True,
                                                   tokenizer=True,
                                                   return_tensors="pt").to(paraphraser.device)
    inp_length=len(tokenized_prompt[0])
    inp_length
    paraphraser_tokenizer.decode(paraphraser.generate(tokenized_prompt, attention_mask=torch.ones_like(tokenized_prompt), max_new_tokens=256)[0][inp_length:], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Your behaviour was unrefined and shallow. You should be able to communicate dissatisfaction in a different manner. Your gesture was inappropriate and it has defamed the reputation of the Hawaii delegation. Apologizing to the people of Hawaii and the DNC is what you ought to do. In case you don't want to apologize, you should step down from your position within the Hawaii Democratic party. There is no courage in this act, simply foolishness. Your actions are reflective of Trump, showing immaturity when faced with disagreement. Perhaps you should join the Republicans as you would probably be more comfortable there. The Democrats do not condone absurdity. She doesn't genuinely love Hawaii. If she did, she wouldn't have caused humiliation to us by her actions.\n"

In [12]:
def generate_new_sentence(sentence, model, tokenizer):
    prompt=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": system_prompt}
    ]
    tokenized_prompt=tokenizer.apply_chat_template(prompt,
                                                   add_generation_prompt=True,
                                                   tokenizer=True,
                                                   return_tensors="pt").to(model.device)
    inp_length=len(tokenized_prompt[0])
    attention_mask=torch.ones_like(tokenized_prompt)
    op=model.generate(tokenized_prompt, attention_mask=attention_mask, max_new_tokens=256)[0][inp_length:]
    return tokenizer.decode(op, skip_special_tokens=True)

In [13]:
def estimate_toxicity(sentence, model, tokenizer):
    tokenized_sentence=tokenizer(sentence, return_tensors="pt").to(model.device)
    logits=model(**tokenized_sentence).logits
    preds=softmax(logits, dim=-1)
    res=np.argmax(preds.half().cpu().detach().numpy(), axis=-1)
    return int(res[0])
    

In [14]:
def check_similarity(old_sentence, new_sentence, model):
    return float(model.predict([old_sentence, new_sentence]))

In [15]:
def generate_new_sample(ex,
                        classifier,
                        classifier_tokenizer,
                        paraphraser,
                        paraphraser_tokenizer,
                        similarity_checker):
    old_sentence=ex["text"]
    old_label=ex["label"]
    new_sentence=generate_new_sentence(old_sentence, paraphraser, paraphraser_tokenizer)
    new_label=estimate_toxicity(old_sentence, classifier, classifier_tokenizer)
    if new_label==old_label:
        return None
    similarity=check_similarity(old_sentence, new_sentence, similarity_checker)
    if similarity<0.4:
        return None
    return {
        "en_toxic_comment": old_sentence if old_label==1 else new_sentence,
        "en_neutral_comment": new_sentence if old_label==1 else old_sentence,
        "sentence_similarity": similarity
    }

In [18]:
def generate_new_samples(ds,
                         classifier,
                         classifier_tokenizer,
                         paraphraser,
                         paraphraser_tokenizer,
                         similarity_checker):
    new_ds=[]
    for ex in tqdm(ds):
        new_ex=generate_new_sample(ex,
                                   classifier,
                                   classifier_tokenizer,
                                   paraphraser,
                                   paraphraser_tokenizer,
                                   similarity_checker)
        if new_ex is None:
            continue
        new_ds.append(new_ex)
    return new_ds

In [ ]:
new_ds=generate_new_samples(ds,
                            classifier,
                            classifier_tokenizer,
                            paraphraser,
                            paraphraser_tokenizer,
                            similarity)

  0%|          | 0/3954 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [21]:
login()

In [ ]:
new_ds=pd.DataFrame(new_ds)
new_ds=Dataset.from_pandas(new_ds, preserve_index=False)
new_ds.push_to_hub("harsha2946/sampl-data")

Signature:
Dataset.from_pandas(
    df: pandas.DataFrame,
    features: Optional[datasets.features.features.Features] = None,
    info: Optional[datasets.info.DatasetInfo] = None,
    split: Optional[datasets.splits.NamedSplit] = None,
    preserve_index: Optional[bool] = None,
) -> 'Dataset'
Docstring:
Convert `pandas.DataFrame` to a `pyarrow.Table` to create a [`Dataset`].

The column types in the resulting Arrow Table are inferred from the dtypes of the `pandas.Series` in the
DataFrame. In the case of non-object Series, the NumPy dtype is translated to its Arrow equivalent. In the
case of `object`, we need to guess the datatype by looking at the Python objects in this Series.

Be aware that Series of the `object` dtype don't carry enough information to always lead to a meaningful Arrow
type. In the case that we cannot infer a type, e.g. because the DataFrame is of length 0 or the Series only
contains `None/nan` objects, the type is set to `null`. This behavior can be avoided by cons